## Linear Regression
An estimator is any model that tries to estimate a variable y from another variable(s) x given pairs of data (x<sub>1</sub>,y<sub>1</sub>),(x<sub>2</sub>,y<sub>2</sub>),...,(x<sub>N</sub>,y<sub>N</sub>)

Regression is when the targets (y) are quantities (not cat vs dog but rather price of a house)

An example would be trying to guess how many people will buy icecream from a shop at any day given the temprature on that day. In this case, the target is the number of icecream people will buy, it is an integer and any value (ex: 14 icecreams) doesn't represent a particular class, so it will be represented as a regression problem. In this case, it turns out that x is also a quantity (temprature); however, this is not necessary and is tackled by approaches other than linear regression.

To estimate the price of the icecream from temperature we gather the data on various days from various different places and get the following data.



We can see that a line (could  also be a curve) could fit the data, but how can we find which line exactly.

#### Line Equation Review
A line in 2D can be parametrized using a slope m and a y-intercept b which would result in it having an equation

$$y = mx+b$$

In the example of the icecream, if the temperature at a day was 30, our prediction for the number of icecreams sold would be m\*30 + b, so we simply need to find m and b.

#### Squared Loss
We need some measurement of how good our line is to be able to find the "best" line, so we will measure the difference between our predictions and the correct values from the data.


Then, we will square all the differences and add them up. This will be the value we try to minimize.

#### Note
The normal equation will probably not work. It is left for you to figure it out and let everyone else know, let's see who figures it out first (it's a tricky issue).




In [1]:
import numpy as np
import pandas as pd

In [2]:
class Linear_Regression():
    '''
    Linear Regression model created using only NumPy
    
    Attributes
    ----------
    weights: np.array of floats
        All the parameters of the model (including bias)
    '''
    def __init__(self):
        self.weights = None
        
    def train(self,data_X,data_y):
        '''
        Train the model using the given data
        
        Parameters
        ----------
        data_X: np.array, shape = (N, num_features)
            Features from the data, each row is one data point. Assumes that a column of ones was added to data_X
        data_y: np.array, shape = (N, num_targets)
            The target values to predict, each row contains the targets for one data point
        '''
        #Insert code here 
        x, y = data_x, data_y
        A = np.linalg.pinv(x)@y
        return A
    
    def predict(self,x_to_predict):
        '''
        Predict using the given value as input
        
        Assumes that self.train(.,.) has been called before calling this method
        
        Parameters
        ----------
        x_to_predict: np.array, shape = (M, num_features)
            A given list of inputs to predict targets for, each row is one input. Assumes that a column of ones was added similar to the training data
        
        Returns
        -------
        np.array of floats, shape = (M, num_targets)
            Predicted values for each input
        '''
        # Insert code here 

### Import the data and remove useless columns

In [3]:
df = pd.read_csv("train.csv")
df.drop(columns=["Id"],inplace=True)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Handle the missing data (NaNs)

In [4]:
df.drop(columns=df.columns[df.isnull().sum().values>200],inplace=True)
df.dropna(inplace=True)
df.isnull().sum().values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

### Replace categorical data (strings) with numerical values

In [5]:
obj_to_replace = df["MSZoning"].dtype

for column in df.columns:
    if df[column].dtype == obj_to_replace:
        uniques = np.unique(df[column].values)
        for idx,item in enumerate(uniques):
            df[column] = df[column].replace(item,idx)
            
df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,3,8450,1,3,3,0,4,0,5,...,0,0,0,0,0,2,2008,8,4,208500
1,20,3,9600,1,3,3,0,2,0,24,...,0,0,0,0,0,5,2007,8,4,181500
2,60,3,11250,1,0,3,0,4,0,5,...,0,0,0,0,0,9,2008,8,4,223500
3,70,3,9550,1,0,3,0,0,0,6,...,272,0,0,0,0,2,2006,8,0,140000
4,60,3,14260,1,0,3,0,2,0,15,...,0,0,0,0,0,12,2008,8,4,250000


### Add the bias column (column of ones)

In [6]:
df["bias"] = np.ones(df.shape[0])
df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,bias
0,60,3,8450,1,3,3,0,4,0,5,...,0,0,0,0,2,2008,8,4,208500,1.0
1,20,3,9600,1,3,3,0,2,0,24,...,0,0,0,0,5,2007,8,4,181500,1.0
2,60,3,11250,1,0,3,0,4,0,5,...,0,0,0,0,9,2008,8,4,223500,1.0
3,70,3,9550,1,0,3,0,0,0,6,...,0,0,0,0,2,2006,8,0,140000,1.0
4,60,3,14260,1,0,3,0,2,0,15,...,0,0,0,0,12,2008,8,4,250000,1.0


### Divide the data into training, testing, X, and y

In [7]:
df = df.sample(frac=1).reset_index(drop=True)
training_df = df[:-100]
val_df = df[-100:]
training_y = training_df["SalePrice"].values
training_X = training_df.drop(columns=["SalePrice"]).values
val_y = val_df["SalePrice"].values
val_X = val_df.drop(columns=["SalePrice"]).values

print(training_X.shape)
print(np.mean(training_y))

(1238, 74)
186773.48384491116


### Train the linear regressor

In [9]:
# Create and fit the model
LR = Linear_Regression().train(training_X,training_y)

# Calculate Mean Absolute Error (Easier to interpret than MSE)
#Insert code here 
MeanAbsoluteError=np.mean(np.abs(training_y-training_X.dot(LR)))
print(MeanAbsoluteError)

19163.83168024689


### Train using the sklearn linear regressor

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Create and fit the model
# Insert code here 
reg = LinearRegression(fit_intercept=False).fit(training_X,training_y)
print(reg.coef_)

# Calculate Mean Absolute Error (Easier to interpret than MSE)
# Insert code here 
mean_absolute_error(training_y, training_X.dot(reg.coef_))

[-1.46637053e+02 -1.83600339e+03  3.48963333e-01  3.74991987e+04
 -1.18915368e+03  4.61328327e+03 -1.18905916e-07  3.43359502e+02
  8.33441598e+03  4.33432539e+02 -4.01837032e+02 -9.22194601e+03
 -4.50425717e+02 -1.51337725e+03  1.22113917e+04  4.91203743e+03
  2.11295553e+02  1.71078714e+01  1.89506906e+03  4.15185036e+03
 -9.69406465e+02  2.87836003e+02  3.63702996e+03  3.02000638e+01
 -7.54047642e+03 -8.59831359e+02  2.26962859e+03 -8.89817023e+03
  3.28039618e+03 -4.14685138e+03 -9.38932379e+02  2.42646353e+00
  8.13250601e+02  5.91716402e+00 -3.32776387e+00  5.01583970e+00
 -5.23105048e+03 -5.26738660e+02  4.19156800e+02 -2.64448596e+02
  1.52346704e+01  1.85863101e+01 -4.49554633e+00  2.93253565e+01
  6.81242537e+03  7.02742915e+02  1.23535502e+03 -8.84925064e+02
 -3.34242829e+03 -2.20665713e+04 -9.08432481e+03  3.84596608e+03
  4.08637174e+03  4.70303559e+03  5.87895688e+02 -1.13302813e+02
 -2.90652446e+02  1.50756630e+04 -4.42042858e-01 -8.34405230e+02
  2.35935455e+03  3.30970

19163.831680246865